In [1]:
import numpy as np
import pandas as pd

In [2]:
df1 = pd.read_csv("C:\\Users\\asus\\NLP\\TIL Dataset.csv")
df1.head(10)

,text,target
0,Musik zu machen liegt uns im Blut. Einfach Kra...,personal finance
1,Roohi’s trailer is haunting and hilarious at t...,family and relationships
2,2012 PLAYBOY JAZZ FESTIVAL FEATURES NEW ARTIST...,music and audio
3,FedEx Office Coupons & Promo Codes. FedEx Offi...,shopping
4,Selecting Words. 26) The Kumbh Mela is known f...,technology and computing
5,Sibley South. The wonderful Sibley Regional Vo...,shopping
6,National Theatre: On Demand. In Schools. Natio...,arts and culture
7,One Shot is Call of Duty: Advanced Warfare’s n...,shopping
8,Pet Loss Books – Words Of Comfort From Pet Los...,family and relationships
9,Store2Vec: DoorDash Recommendations with Mitch...,technology and computing


In [6]:
df1[df1.target == 'pets']

,text,target
39,The Academy Museum of Motion Pictures Opening ...,pets
141,Employee Stock Purchase Plans. Get information...,pets
429,Another Bird Blog. The publication of new book...,pets
490,5 Tips on How To Effectively Use Tanning Drops...,pets
520,what are they carrying. Emmy Rossum: Christian...,pets
...,...,...
246409,birds now - birds and parrots for sale and ado...,pets
246410,the 2hr aquarist the ultimate guide to m...,pets
246449,pipa | racing pigeons for sale | pigeon news |...,pets
246624,"\tveterinary information networkÂ®, inc. - vin...",pets


In [7]:
import spacy
import random

nlp = spacy.load('en_core_web_lg')

C:\Users\asus\AppData\Local\Programs\Python\Python39\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
def get_synonyms_spacy(word):
    word_vector = nlp.vocab[word].vector
    most_similar = nlp.vocab.vectors.most_similar(word_vector.reshape(1, -1), n=5)
    synonyms = [nlp.vocab.strings[similar] for similar in most_similar[0][0]]
    if word in synonyms:
        synonyms.remove(word)
    return synonyms

def synonym_replacement_spacy(text, n=2):
    doc = nlp(text)
    augmented_texts = []
    for _ in range(n):
        augmented_text = [token.text for token in doc]
        for token in doc:
            if random.random() < 0.15:
                synonyms = get_synonyms_spacy(token.text)
                if synonyms:
                    augmented_text[token.i] = random.choice(synonyms)
        augmented_texts.append(' '.join(augmented_text))
    return augmented_texts

In [9]:
def random_deletion_spacy(text, n=2, p=0.15):
    doc = nlp(text)
    augmented_texts = []
    for _ in range(n):
        augmented_text = [token.text for token in doc if random.random() > p]
        augmented_texts.append(' '.join(augmented_text))
    return augmented_texts

In [10]:
def text_rotation_spacy(text, n=2):
    augmented_texts = []
    sentences = list(nlp(text).sents)
    for _ in range(n):
        random.shuffle(sentences)
        augmented_texts.append(' '.join([sent.text for sent in sentences]))
    return augmented_texts

In [11]:
from googletrans import Translator

def back_translate(text, target_lang, source_lang="en"):
    translator = Translator()
    translated_text = translator.translate(text, dest=target_lang, src=source_lang).text
    back_translated_text = translator.translate(translated_text, dest=source_lang, src=target_lang).text
    return back_translated_text

In [13]:
desired_category = 'pets'

selected_rows = df1[df1['target'] == desired_category]
augmented_texts = selected_rows['text'].apply(random_deletion_spacy)

augmented_df = pd.DataFrame({
    'text': augmented_texts.explode(),
    'target': desired_category
})

df = pd.concat([df1, augmented_df], ignore_index=True)

In [14]:
df[df.target == 'pets'].count()

text      633
target    633
dtype: int64